In [1]:
!pip install geopy
!pip install folium
import pandas as pd
import math
import random
import folium
from geopy.distance import geodesic

C:\Users\Promi\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Load the charging stations data
charging_stations_df = pd.read_csv("Electric_Vehicle_Charging_Stations.csv")

# Load the places data
places_df = pd.read_csv("coordinates.csv")

# Concatenate "Station Name" and "Street Address" into a new column "Charging Stations"
charging_stations_df["Charging Stations"] = charging_stations_df["Station Name"] + ", " + charging_stations_df["Street Address"]
charging_stations_df = charging_stations_df.drop(columns=["Station Name", "Street Address"])

# Rename the "Places" column to "Locations" in places_df
places_df = places_df.rename(columns={"Places": "Locations"})

# Display the resulting DataFrames
print("Charging Stations DataFrame:")
print(charging_stations_df.head())
print("\nPlaces DataFrame:")
print(places_df.head())

Charging Stations DataFrame:
           City              Access Days Time EV Level1 EVSE Num  \
0        Darien                24 hours daily               NONE   
1       Meriden  24 hours, for Tesla use only               NONE   
2  Beacon Falls                24 hours daily               NONE   
3  Old Saybrook                24 hours daily               NONE   
4     Fairfield                24 hours daily               NONE   

  EV Level2 EVSE Num EV DC Fast Count  Longitude   Latitude  \
0                  2             NONE -73.476469  41.072882   
1               NONE                8 -72.773473  41.527367   
2                  1             NONE -73.065583  41.445481   
3                  2             NONE -72.382500  41.310278   
4                  2             NONE -73.264511  41.143125   

                                   Charging Stations  
0                    BMW OF DARIEN, 138-142 Ledge Rd  
1        Dunkin’ - Tesla Supercharger, 893 E Main St  
2  Town of Beacon 

In [3]:
# Create a distance matrix
distance_matrix = pd.DataFrame(index=charging_stations_df["Charging Stations"], columns=places_df["Locations"])

for idx, row in charging_stations_df.iterrows():
    for col in places_df["Locations"]:
        station_coords = (row["Latitude"], row["Longitude"])
        place_coords = (places_df.loc[places_df["Locations"] == col, "Latitude"].values[0],
                        places_df.loc[places_df["Locations"] == col, "Longitude"].values[0])
        distance = geodesic(station_coords, place_coords).miles
        distance_matrix.at[row["Charging Stations"], col] = distance

In [4]:

# Create a distance matrix with distance for qualifying charging stations
qualifying_charging_stations = charging_stations_df[
    (charging_stations_df["Access Days Time"].str.contains("24 hours")) &
    (
        (pd.to_numeric(charging_stations_df["EV Level2 EVSE Num"], errors='coerce') > 0) |
        (pd.to_numeric(charging_stations_df["EV DC Fast Count"], errors='coerce') > 0)
    )]

qualifying_distance_matrix = pd.DataFrame(index=qualifying_charging_stations["Charging Stations"], columns=places_df["Locations"])

for idx, row in qualifying_charging_stations.iterrows():
    for col in places_df["Locations"]:
        qualifying_distance_matrix.at[row["Charging Stations"], col] = distance_matrix.at[row["Charging Stations"], col]
print(qualifying_distance_matrix)

Locations                                          Connecticut State Capitol  \
Charging Stations                                                              
BMW OF DARIEN, 138-142 Ledge Rd                                    63.075025   
Dunkin’ - Tesla Supercharger, 893 E Main St                        17.016899   
Town of Beacon Falls - Commuter Lot, 105 N Main St                 29.634424   
OLD SAYBROOK VW, 319 Middlesex Turnpike                            34.974589   
Fairfield Rail Station, 80 Mill Plain Rd                           52.451869   
...                                                                      ...   
Yankee Doodle, 10 Burnell Boulevard                                58.620315   
777 MAIN ST, 71 Asylum St                                            0.43666   
Norwich Public Utilities, 173 N Main St                            35.955034   
Fairfield Plaza, 116 Danbury Rd                                    40.647661   
Eastern Connecticut State University - S

In [5]:
# Create a location distance matrix
location_distance_matrix = pd.DataFrame(index=places_df["Locations"], columns=places_df["Locations"])

for idx1, row1 in places_df.iterrows():
    for idx2, row2 in places_df.iterrows():
        location1_coords = (row1["Latitude"], row1["Longitude"])
        location2_coords = (row2["Latitude"], row2["Longitude"])
        distance = geodesic(location1_coords, location2_coords).miles
        location_distance_matrix.at[row1["Locations"], row2["Locations"]] = distance

# Display the location distance matrix
print("Location Distance Matrix:")
print(location_distance_matrix)

Location Distance Matrix:
Locations                                   Connecticut State Capitol  \
Locations                                                               
Connecticut State Capitol                                         0.0   
Pratt & Whitney Stadium at Rentschler Field                  3.305513   
Talcott Mountain State Park                                  7.459013   
New England Air Museum                                      12.670356   
Phelps-Hatheway House & Garden                              14.984078   
Connecticut Trolley Museum                                  12.426005   
Shenipsit State Forest                                      19.631927   
University of Connecticut                                   22.325602   
Patriots Park                                               19.411611   
Wadsworth Falls State Park                                  15.691763   
Hubbard Park                                                16.901365   
Wharton Brook State Park 

In [6]:
# Create a distance matrix for charging station to charging station
charging_station_distance_matrix = pd.DataFrame(index=charging_stations_df["Charging Stations"], columns=charging_stations_df["Charging Stations"])

for idx1, row1 in charging_stations_df.iterrows():
    for idx2, row2 in charging_stations_df.iterrows():
        station1_coords = (row1["Latitude"], row1["Longitude"])
        station2_coords = (row2["Latitude"], row2["Longitude"])
        distance = geodesic(station1_coords, station2_coords).miles
        charging_station_distance_matrix.at[row1["Charging Stations"], row2["Charging Stations"]] = distance

print("Charging Station Distance Matrix:")
print(charging_station_distance_matrix)

Charging Station Distance Matrix:
Charging Stations                                  BMW OF DARIEN, 138-142 Ledge Rd  \
Charging Stations                                                                    
BMW OF DARIEN, 138-142 Ledge Rd                                                0.0   
Dunkin’ - Tesla Supercharger, 893 E Main St                              48.188174   
Town of Beacon Falls - Commuter Lot, 105 N Main St                       33.450536   
OLD SAYBROOK VW, 319 Middlesex Turnpike                                  59.331846   
Fairfield Rail Station, 80 Mill Plain Rd                                 12.078196   
...                                                                            ...   
Lee Partyka Chevrolet Mazda Isuzu, 200 Skiff St                          36.054234   
777 MAIN ST, 71 Asylum St                                                63.466218   
Norwich Public Utilities, 173 N Main St                                  79.945681   
Fairfield Plaza, 116

### NEAREST INSERTION ALGORITHM

In [7]:
def find_nearest_location(current_tour, location_distance_matrix, qualifying_distance_matrix, places_df, charging_stations_df):
    print("Finding nearest location...")
    nearest_location = None
    min_distance = float('inf')

    for location in places_df["Locations"]:
        if location not in current_tour:
            for city in current_tour:
                if city in places_df["Locations"].values:
                    distance = location_distance_matrix.at[city, location]
                elif city in charging_stations_df["Charging Stations"].values:
                    # Use qualifying_distance_matrix for charging stations
                    distance = qualifying_distance_matrix.at[city, location]
                else:
                    raise ValueError(f"Invalid city: {city}")
                if distance < min_distance:
                    nearest_location = location
                    min_distance = distance

    print("Nearest location found:", nearest_location)
    return nearest_location

def find_nearest_charging_station(current_tour, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df):
    print("Finding nearest charging station...")
    nearest_station = None
    min_distance = float('inf')

    # This function finds the nearest unvisited charging station from the current tour.
    for station in qualifying_distance_matrix.index:
        if station not in current_tour:
            # For each unvisited charging station, it calculates the distance from each city in the current tour to that charging station.
            for city in current_tour:
                if city in places_df["Locations"].values:
                    # If the city in the current tour is a location, it uses the qualifying_distance_matrix to get the distance.
                    distance = qualifying_distance_matrix.at[station, city]
                elif city in charging_stations_df["Charging Stations"].values:
                    # If the city in the current tour is a charging station, it uses the charging_station_distance_matrix to get the distance.
                    distance = charging_station_distance_matrix.at[station, city]
                else:
                    raise ValueError(f"Invalid city: {city}")

                if distance < min_distance:
                    nearest_station = station
                    min_distance = distance

    # It keeps track of the charging station with the minimum distance from the current tour.
    # The function returns the nearest unvisited charging station.
    if nearest_station is not None:
        print("Nearest charging station found:", nearest_station)
    else:
        print("No suitable charging station found.")
    return nearest_station

# Note: The qualifying_distance_matrix has charging stations in its index position and locations in its column position.
# The charging_station_distance_matrix has distances between charging stations.

def insert_nearest_charging_station(nearest_station, current_tour, location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df):
    print("Inserting nearest charging station...")
    best_insertion = None
    min_increase = float('inf')

    for i in range(len(current_tour)):
        prev_city = current_tour[i]
        next_i = (i + 1) % len(current_tour)
        next_city = current_tour[next_i]

        if prev_city in places_df["Locations"].values and next_city in places_df["Locations"].values:
            if nearest_station in qualifying_distance_matrix.index and prev_city in qualifying_distance_matrix.columns and next_city in qualifying_distance_matrix.columns:
                increase = qualifying_distance_matrix.at[nearest_station, prev_city] + \
                           qualifying_distance_matrix.at[nearest_station, next_city] - \
                           location_distance_matrix.at[prev_city, next_city]
            else:
                continue
        elif prev_city in places_df["Locations"].values and next_city in charging_stations_df["Charging Stations"].values:
            if nearest_station in charging_station_distance_matrix.index and prev_city in qualifying_distance_matrix.columns and next_city in charging_station_distance_matrix.columns:
                increase = qualifying_distance_matrix.at[nearest_station, prev_city] + \
                           charging_station_distance_matrix.at[nearest_station, next_city] - \
                           qualifying_distance_matrix.at[next_city, prev_city]
            else:
                continue
        elif prev_city in charging_stations_df["Charging Stations"].values and next_city in places_df["Locations"].values:
            if nearest_station in charging_station_distance_matrix.index and prev_city in charging_station_distance_matrix.columns and next_city in qualifying_distance_matrix.columns:
                increase = charging_station_distance_matrix.at[nearest_station, prev_city] + \
                           qualifying_distance_matrix.at[nearest_station, next_city] - \
                           qualifying_distance_matrix.at[prev_city, next_city]
            else:
                continue
        else:  # Both prev_city and next_city are charging stations
            if nearest_station in charging_station_distance_matrix.index and prev_city in charging_station_distance_matrix.columns and next_city in charging_station_distance_matrix.columns:
                increase = charging_station_distance_matrix.at[nearest_station, prev_city] + \
                           charging_station_distance_matrix.at[nearest_station, next_city] - \
                           charging_station_distance_matrix.at[prev_city, next_city]
            else:
                continue

        if increase < min_increase:
            min_increase = increase
            best_insertion = i + 1

    if best_insertion is not None:
        current_tour.insert(best_insertion, nearest_station)
        print("Nearest charging station inserted into the tour.")
    else:
        print("No suitable insertion point found for the nearest charging station.")

def insert_nearest_location(nearest_location, current_tour, location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df):
    print("Inserting nearest location...")
    best_insertion = None
    min_increase = float('inf')

    for i in range(len(current_tour)):
        prev_city = current_tour[i]
        next_i = (i + 1) % len(current_tour)
        next_city = current_tour[next_i]

        if prev_city in places_df["Locations"].values and next_city in places_df["Locations"].values:
            increase = location_distance_matrix.at[prev_city, nearest_location] + \
                       location_distance_matrix.at[nearest_location, next_city] - \
                       location_distance_matrix.at[prev_city, next_city]
        elif prev_city in places_df["Locations"].values and next_city in charging_stations_df["Charging Stations"].values:
            # Use qualifying_distance_matrix with charging station as index and location as column
            increase = qualifying_distance_matrix.at[next_city, prev_city] + \
                       qualifying_distance_matrix.at[next_city, nearest_location] - \
                       qualifying_distance_matrix.at[next_city, prev_city]
        elif prev_city in charging_stations_df["Charging Stations"].values and next_city in places_df["Locations"].values:
            # Use qualifying_distance_matrix with charging station as index and location as column
            increase = qualifying_distance_matrix.at[prev_city, nearest_location] + \
                       qualifying_distance_matrix.at[prev_city, next_city] - \
                       qualifying_distance_matrix.at[prev_city, next_city]
        else:  # Both prev_city and next_city are charging stations
            if nearest_location in charging_station_distance_matrix.columns:
                increase = charging_station_distance_matrix.at[prev_city, nearest_location] + \
                           charging_station_distance_matrix.at[nearest_location, next_city] - \
                           charging_station_distance_matrix.at[prev_city, next_city]
            else:
                # If nearest_location is not in charging_station_distance_matrix, assign a large value to increase
                increase = float('inf')

        if increase < min_increase:
            min_increase = increase
            best_insertion = i + 1

    current_tour.insert(best_insertion, nearest_location)
    print("Nearest location inserted into the tour.")

def calculate_battery_discharge_rate(distance, speed, Cd, A, Crr, m, g, rho, lambda_, alpha):
    # Calculate the battery discharge rate based on vehicle parameters
    F_drag = 0.5 * rho * Cd * A * (speed ** 2)
    F_rolling = Crr * m * g
    F_total = F_drag + F_rolling
    P_total = F_total * speed
    P_regen = alpha * P_total
    P_net = P_total - P_regen
    energy_consumed = P_net * (distance / speed)
    discharge_rate = energy_consumed / 3600  # Convert from Joules to kWh
    return discharge_rate

def print_final_tour(tour, places_df, charging_stations_df, total_distance):
    print("Final Tour:")
    for city in tour:
        if city in places_df["Locations"].values:
            print(f"Nearest Location: {city}")
        else:
            print(f"Nearest Charging Station: {city}")
    print(f"Best Cost (Total Distance): {total_distance}")

def get_distance(city1, city2, location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df):
    if city1 in places_df["Locations"].values and city2 in places_df["Locations"].values:
        return location_distance_matrix.at[city1, city2]
    elif city1 in places_df["Locations"].values and city2 in charging_stations_df["Charging Stations"].values:
        return qualifying_distance_matrix.at[city2, city1]
    elif city1 in charging_stations_df["Charging Stations"].values and city2 in places_df["Locations"].values:
        return qualifying_distance_matrix.at[city1, city2]
    elif city1 in charging_stations_df["Charging Stations"].values and city2 in charging_stations_df["Charging Stations"].values:
        return charging_station_distance_matrix.at[city1, city2]
    else:
        raise KeyError(f"Invalid cities: {city1}, {city2}")
def nearest_insertion_algorithm(location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df, source_city, vehicle_params, charging_params, iteration_count=0):
    current_tour = [source_city]
    visited_cities = set(current_tour)
    current_charge = charging_params['max_charge']
    total_distance = 0
    speed_dict = {}

    Cd, A, Crr, m, g, rho, lambda_, alpha = vehicle_params.values()
    level2_charge_rate, dc_fast_charge_rate, min_charge, max_charge = charging_params.values()


    while len(set(current_tour) & set(places_df["Locations"])) < len(places_df["Locations"]):
        nearest_location = find_nearest_location(current_tour, location_distance_matrix, qualifying_distance_matrix, places_df, charging_stations_df)
        distance_to_next_location = location_distance_matrix.at[current_tour[-1], nearest_location]

        speed = 60  # Random speed between 50 and 80 miles/h
        #speed_dict[(current_tour[-1], nearest_location)] = speed  # Store the speed in the speed_dict
        #speed_dict[(nearest_location, current_tour[-1])] = speed  # Also store the speed for the reverse direction
        print(f"Speed stored in speed_dict: {current_tour[-1]} -> {nearest_location}: {speed} miles/h")
        print(f"Speed stored in speed_dict: {nearest_location} -> {current_tour[-1]}: {speed} miles/h")
        print(f"Speed stored in speed_dict: {current_tour[-1]} -> {nearest_location}: {speed} miles/h")
        discharge_rate = calculate_battery_discharge_rate(distance_to_next_location, speed, Cd, A, Crr, m, g, rho, lambda_, alpha)
        required_charge_to_next_location = discharge_rate

        print(f"Current charge: {current_charge}, Required charge to next location: {required_charge_to_next_location}")

        if current_charge - required_charge_to_next_location >= min_charge:
            insert_nearest_location(nearest_location, current_tour, location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df)
            visited_cities.add(nearest_location)
            total_distance += distance_to_next_location
            current_charge -= required_charge_to_next_location

            print("Updated Tour:")
            for city in current_tour:
                print(f"Nearest Location: {city}")
            print(f"Total Distance: {total_distance}")
            print(f"Updated Charge: {current_charge}")
            print(f"Speed: {speed} miles/h")


            # Check if charge level is below 60% and find the nearest charging station
            if current_charge < max_charge * 0.6:
                print("Warning: Charge level below 60%. Looking for the nearest charging station.")
                nearest_station = find_nearest_charging_station(current_tour, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df)

                if nearest_station is not None:
                    insert_nearest_charging_station(nearest_station, current_tour, location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df)
                    visited_cities.add(nearest_station)
                    distance_to_station = get_distance(current_tour[-2], nearest_station, location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df)
                    current_charge = max_charge
                    print("Total Distance After Charging Station Insertion:", total_distance)

        else:
            print(f"Impossible route encountered after {iteration_count} iterations. Restarting with a different source city.")
            new_source_city = places_df.loc[places_df["Locations"] != source_city, "Locations"].sample().values[0]
            return nearest_insertion_algorithm(location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df, new_source_city, vehicle_params, charging_params, iteration_count + 1)

    print(f"Nearest Insertion Algorithm completed after {iteration_count} iterations.")
    print("Tour:")
    for city in current_tour:
        if city in places_df["Locations"].values:
            print(f"Nearest Location: {city}")
        else:
            print(f"Nearest Charging Station: {city}")

    if all(location in current_tour for location in places_df["Locations"]):
        print("Feasible route found.")
        print_final_tour(current_tour, places_df, charging_stations_df, total_distance)
        return current_tour, total_distance, speed_dict
    else:
        print(f"Impossible route encountered after {iteration_count} iterations. Restarting with a different source city.")
        new_source_city = places_df.loc[places_df["Locations"] != source_city, "Locations"].sample().values[0]
        return nearest_insertion_algorithm(location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df, new_source_city, vehicle_params, charging_params, iteration_count + 1)

vehicle_params = {
    'Cd': 0.3,
    'A': 2.2,
    'Crr': 0.01,
    'm': 1500,
    'g': 9.81,
    'rho': 1.225,
    'lambda_': 0.80,
    'alpha': 0.25
}

charging_params = {
    'level2_charge_rate': 7,
    'dc_fast_charge_rate': 50,
    'min_charge': 20,  # 20% of 100 kWh battery capacity
    'max_charge': 90  # 90% of 100 kWh battery capacity
} 

In [8]:
source_city = "Yale University"
tour, total_distance, speed= nearest_insertion_algorithm(location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df, source_city, vehicle_params, charging_params)

Finding nearest location...
Nearest location found: Sleeping Giant State Park
Speed stored in speed_dict: Yale University -> Sleeping Giant State Park: 60 miles/h
Speed stored in speed_dict: Sleeping Giant State Park -> Yale University: 60 miles/h
Speed stored in speed_dict: Yale University -> Sleeping Giant State Park: 60 miles/h
Current charge: 90, Required charge to next location: 2.4439899817787563
Inserting nearest location...
Nearest location inserted into the tour.
Updated Tour:
Nearest Location: Yale University
Nearest Location: Sleeping Giant State Park
Total Distance: 7.320760031537976
Updated Charge: 87.55601001822124
Speed: 60 miles/h
Finding nearest location...
Nearest location found: Wharton Brook State Park
Speed stored in speed_dict: Sleeping Giant State Park -> Wharton Brook State Park: 60 miles/h
Speed stored in speed_dict: Wharton Brook State Park -> Sleeping Giant State Park: 60 miles/h
Speed stored in speed_dict: Sleeping Giant State Park -> Wharton Brook State Par

In [11]:
def remove_redundant_charging_stations(final_tour, places_df, charging_stations_df, vehicle_params, charging_params, speed_dict):
    filtered_tour = []
    current_charge = charging_params['max_charge']
    total_distance = 0
    Cd, A, Crr, m, g, rho, lambda_, alpha = vehicle_params.values()
    level2_charge_rate, dc_fast_charge_rate, min_charge, max_charge = charging_params.values()
    default_speed = 40  # Assume a default speed of 60 miles/h if speed is not found in speed_dict

    # Calculate the maximum range based on vehicle parameters and charging parameters
    max_range = (max_charge - min_charge) / (0.5 * rho * Cd * A * (default_speed / 0.6213711922) ** 2 + Crr * m * g) * 3600  # in miles

    for i, city in enumerate(final_tour):
        if city in places_df["Locations"].values:
            filtered_tour.append(city)
            if i < len(final_tour) - 1:  # Check if there is a next city in the tour
                next_city = final_tour[i + 1]
                distance_to_next_location = get_distance(city, next_city, location_distance_matrix,
                                                         qualifying_distance_matrix, charging_station_distance_matrix,
                                                         places_df, charging_stations_df)

                # Check for both orders of the cities in the speed_dict
                if (city, next_city) in speed_dict:
                    speed = speed_dict[(city, next_city)]
                elif (next_city, city) in speed_dict:
                    speed = speed_dict[(next_city, city)]
                else:
                    # If speed is not found in speed_dict, use the default speed
                    speed = default_speed
                    print(f"Warning: Speed not found for the path between {city} and {next_city}. Using default speed of {default_speed} miles/h.")

                discharge_rate = calculate_battery_discharge_rate(distance_to_next_location, speed, Cd, A, Crr, m, g,
                                                                  rho, lambda_, alpha)
                current_charge -= discharge_rate
                print(f"Charge level after visiting {city}: {current_charge:.2f} (Lost {discharge_rate:.2f})")

                # Check if the current charge falls below 60% of the maximum charge
                if current_charge < 0.6 * max_charge:
                    # Check if the next city is a charging station in the final_tour
                    if next_city in charging_stations_df["Charging Stations"].values and next_city in final_tour:
                        # Include the charging station in the filtered_tour
                        filtered_tour.append(next_city)
                        current_charge = max_charge
                        print(f"Charge level after visiting {next_city}: {current_charge:.2f} (Added {max_charge - current_charge:.2f})")

            total_distance += distance_to_next_location

        elif city in charging_stations_df["Charging Stations"].values:
            # Check if the charging station needs to be removed
            if i < len(final_tour) - 1:
                next_city = final_tour[i + 1]
                distance_to_next_location = get_distance(city, next_city, location_distance_matrix,
                                                         qualifying_distance_matrix, charging_station_distance_matrix,
                                                         places_df, charging_stations_df)

                # Check for both orders of the cities in the speed_dict
                if (city, next_city) in speed_dict:
                    speed = speed_dict[(city, next_city)]
                elif (next_city, city) in speed_dict:
                    speed = speed_dict[(next_city, city)]
                else:
                    # If speed is not found in speed_dict, use the default speed
                    speed = default_speed
                    print(f"Warning: Speed not found for the path between {city} and {next_city}. Using default speed of {default_speed} miles/h.")

                discharge_rate = calculate_battery_discharge_rate(distance_to_next_location, speed, Cd, A, Crr, m, g,
                                                                  rho, lambda_, alpha)
                remaining_charge = current_charge - discharge_rate

                # Check if the remaining charge is below 60% of the maximum charge
                if remaining_charge < 0.6 * max_charge:
                    # The charging station is needed, so include it in the filtered_tour
                    filtered_tour.append(city)
                    current_charge = max_charge
                    print(f"Charge level after visiting {city}: {current_charge:.2f} (Added {max_charge - remaining_charge:.2f})")

            # If the charging station is the last city in the tour, include it in the filtered_tour
            if i == len(final_tour) - 1:
                filtered_tour.append(city)

    print("Optimized Final Tour:")
    for city in filtered_tour:
        if city in places_df["Locations"].values:
            print(f"Nearest Location: {city}")
        else:
            print(f"Nearest Charging Station: {city}")
    print(f"Best Cost (Total Distance): {total_distance}")

    return filtered_tour

In [12]:
# Call the function with the optimized tour
filtered_tour = remove_redundant_charging_stations(tour, places_df, charging_stations_df, vehicle_params, charging_params, speed)

Charge level after visiting Yale University: 88.56 (Lost 1.44)
Charge level after visiting Wharton Brook State Park: 87.15 (Lost 1.41)
Charge level after visiting Hubbard Park: 85.86 (Lost 1.29)
Charge level after visiting Wadsworth Falls State Park: 83.25 (Lost 2.61)
Charge level after visiting Pratt & Whitney Stadium at Rentschler Field: 80.58 (Lost 2.67)
Charge level after visiting Patriots Park: 76.80 (Lost 3.78)
Charge level after visiting Rotary Park & Bandstand: 70.08 (Lost 6.72)
Charge level after visiting Fort Trumbull State Park: 68.95 (Lost 1.13)
Charge level after visiting Mystic Seaport Museum: 66.64 (Lost 2.31)
Charge level after visiting Mohegan Park: 64.43 (Lost 2.20)
Charge level after visiting Windham Textile & History Museum: 63.25 (Lost 1.18)
Charge level after visiting University of Connecticut: 59.67 (Lost 3.58)
Charge level after visiting Connecticut State Capitol: 58.44 (Lost 1.23)
Charge level after visiting Talcott Mountain State Park: 56.35 (Lost 2.09)
Charge

In [13]:
def create_folium_graph(filtered_tour, places_df, charging_stations_df):
    # Create a map centered on the mean latitude and longitude of all locations
    map_center = (places_df["Latitude"].mean(), places_df["Longitude"].mean())
    folium_map = folium.Map(location=map_center, zoom_start=8)

    # Add markers for each location in the optimized tour
    for city in filtered_tour:
        if city in places_df["Locations"].values:
            location_info = places_df[places_df["Locations"] == city]
            folium.Marker(
                location=[location_info["Latitude"].values[0], location_info["Longitude"].values[0]],
                popup=city,
                icon=folium.Icon(color='blue', icon='info-sign')
            ).add_to(folium_map)
        else:
            station_info = charging_stations_df[charging_stations_df["Charging Stations"] == city]
            folium.Marker(
                location=[station_info["Latitude"].values[0], station_info["Longitude"].values[0]],
                popup=city,
                icon=folium.Icon(color='green', icon='info-sign')
            ).add_to(folium_map)

    # Connect the locations and charging stations in the order of the optimized tour
    locations = []
    for city in filtered_tour:
        if city in places_df["Locations"].values:
            location_info = places_df[places_df["Locations"] == city]
            locations.append([location_info["Latitude"].values[0], location_info["Longitude"].values[0]])
        else:
            station_info = charging_stations_df[charging_stations_df["Charging Stations"] == city]
            locations.append([station_info["Latitude"].values[0], station_info["Longitude"].values[0]])

    # Add the first location again to complete the cycle
    locations.append(locations[0])

    folium.PolyLine(
        locations=locations,
        color='red',
        weight=2,
        opacity=1
    ).add_to(folium_map)

    return folium_map

# Call the function with the optimized tour
#optimized_tour = remove_redundant_charging_stations(tour, places_df, charging_stations_df, vehicle_params, charging_params)


In [14]:
optimized_folium_map = create_folium_graph(filtered_tour, places_df, charging_stations_df)
optimized_folium_map

These values represent various parameters and coefficients used in vehicle dynamics and aerodynamics calculations. Let's go through each one:

1. 'Cd': 0.3 stands for the coefficient of drag, which is a dimensionless quantity that represents the aerodynamic drag of an object.
2. 'A': 2.2 represents the frontal area of the vehicle in square meters (m^2).
3. 'Crr': 0.01 stands for the coefficient of rolling resistance, which is a dimensionless quantity that represents the resistance to motion of the vehicle due to friction between the tires and the road surface.
4. 'm': 1500 represents the mass of the vehicle in kilograms (kg).
5. 'g': 9.81 represents the acceleration due to gravity in meters per second squared (m/s^2).
6. 'rho': 1.225 represents the density of air in kilograms per cubic meter (kg/m^3).
7. 'lambda_': 0.80 is a coefficient related to the efficiency of the vehicle's powertrain.
8. 'alpha': 0.25 is a coefficient related to the road gradient or slope.


In [ ]:
import folium

def create_html_graph(optimized_tour, places_df, charging_stations_df):
    # Create a map centered on the mean latitude and longitude of all locations
    map_center = (places_df["Latitude"].mean(), places_df["Longitude"].mean())
    folium_map = folium.Map(location=map_center, zoom_start=8)

    # Add markers for each location in the optimized tour
    for city in filtered_tour:
        if city in places_df["Locations"].values:
            location_info = places_df[places_df["Locations"] == city]
            folium.Marker(
                location=[location_info["Latitude"].values[0], location_info["Longitude"].values[0]],
                popup=city,
                icon=folium.Icon(color='blue', icon='info-sign')
            ).add_to(folium_map)
        else:
            station_info = charging_stations_df[charging_stations_df["Charging Stations"] == city]
            folium.Marker(
                location=[station_info["Latitude"].values[0], station_info["Longitude"].values[0]],
                popup=city,
                icon=folium.Icon(color='green', icon='info-sign')
            ).add_to(folium_map)

    # Connect the locations and charging stations in the order of the optimized tour
    locations = []
    for city in filtered_tour:
        if city in places_df["Locations"].values:
            location_info = places_df[places_df["Locations"] == city]
            locations.append([location_info["Latitude"].values[0], location_info["Longitude"].values[0]])
        else:
            station_info = charging_stations_df[charging_stations_df["Charging Stations"] == city]
            locations.append([station_info["Latitude"].values[0], station_info["Longitude"].values[0]])

    # Add the first location again to complete the cycle
    locations.append(locations[0])

    folium.PolyLine(
        locations=locations,
        color='red',
        weight=2,
        opacity=1
    ).add_to(folium_map)

    # Save the map as an HTML file
    folium_map.save("optimized_tour.html")

# Call the function with the optimized tour
create_html_graph(filtered_tour, places_df, charging_stations_df)

### Nearest Neighbor Algorithm

In [15]:
import random

def calculate_battery_discharge_rate(distance, speed, Cd, A, Crr, m, g, rho, lambda_, alpha):
    # Calculate the battery discharge rate based on vehicle parameters
    F_drag = 0.5 * rho * Cd * A * (speed ** 2)
    F_rolling = Crr * m * g
    F_total = F_drag + F_rolling
    P_total = F_total * speed
    P_regen = alpha * P_total
    P_net = P_total - P_regen
    energy_consumed = P_net * (distance / speed)
    discharge_rate = energy_consumed / 3600  # Convert from Joules to kWh
    return discharge_rate

def print_final_tour(tour, places_df, charging_stations_df, total_distance):
    print("Final Tour:")
    for city in tour:
        if city in places_df["Locations"].values:
            print(f"Nearest Location: {city}")
        else:
            print(f"Nearest Charging Station: {city}")
    print(f"Best Cost (Total Distance): {total_distance}")

def get_distance(city1, city2, location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df):
    if city1 in places_df["Locations"].values and city2 in places_df["Locations"].values:
        return location_distance_matrix.at[city1, city2]
    elif city1 in places_df["Locations"].values and city2 in charging_stations_df["Charging Stations"].values:
        if city2 in qualifying_distance_matrix.index:
            return qualifying_distance_matrix.at[city2, city1]
        else:
            return float('inf')
    elif city1 in charging_stations_df["Charging Stations"].values and city2 in places_df["Locations"].values:
        if city1 in qualifying_distance_matrix.index:
            return qualifying_distance_matrix.at[city1, city2]
        else:
            return float('inf')
    elif city1 in charging_stations_df["Charging Stations"].values and city2 in charging_stations_df["Charging Stations"].values:
        if city1 in charging_station_distance_matrix.index and city2 in charging_station_distance_matrix.columns:
            return charging_station_distance_matrix.at[city1, city2]
        else:
            return float('inf')
    else:
        raise KeyError(f"Invalid cities: {city1}, {city2}")

In [16]:
def nearest_neighbor_algorithm(location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df, source_city, vehicle_params, charging_params, speed_dict, iteration_count=0):
    current_tour = [source_city]
    visited_cities = set(current_tour)
    current_charge = charging_params['max_charge']
    total_distance = 0

    Cd, A, Crr, m, g, rho, lambda_, alpha = vehicle_params.values()
    level2_charge_rate, dc_fast_charge_rate, min_charge, max_charge = charging_params.values()

    while len(visited_cities) < len(places_df["Locations"]):
        nearest_location = None
        min_distance = float('inf')

        for location in places_df["Locations"]:
            if location not in visited_cities:
                distance = get_distance(current_tour[-1], location, location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df)
                if distance < min_distance:
                    nearest_location = location
                    min_distance = distance

        distance_to_next_location = min_distance

        speed = 40  # Generate random speed between 50 and 80 miles/h
        #speed_dict[(current_tour[-1], nearest_location)] = speed  # Store the speed in the speed_dict
        #speed_dict[(nearest_location, current_tour[-1])] = speed  # Also store the speed for the reverse direction
        print(f"Speed from speed_dict: {current_tour[-1]} -> {nearest_location}: {speed} miles/h")
        discharge_rate = calculate_battery_discharge_rate(distance_to_next_location, speed, Cd, A, Crr, m, g, rho, lambda_, alpha)
        required_charge_to_next_location = discharge_rate

        print(f"Current charge: {current_charge}, Required charge to next location: {required_charge_to_next_location}")

        if current_charge - required_charge_to_next_location >= min_charge:
            current_tour.append(nearest_location)
            visited_cities.add(nearest_location)
            total_distance += distance_to_next_location
            current_charge -= required_charge_to_next_location

            print("Updated Tour:")
            for city in current_tour:
                print(f"Nearest Location: {city}")
            print(f"Total Distance: {total_distance}")
            print(f"Updated Charge: {current_charge}")
            print(f"Speed: {speed} miles/h")


            # Check if charge level is below 60% and find the nearest charging station
            if current_charge < max_charge * 0.6:
                print("Warning: Charge level below 60%. Looking for the nearest charging station.")
                nearest_station = None
                min_distance = float('inf')

                for station in charging_stations_df["Charging Stations"]:
                    if station not in visited_cities:
                        distance = get_distance(current_tour[-1], station, location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df)
                        if distance < min_distance:
                            nearest_station = station
                            min_distance = distance

                if nearest_station is not None:
                    current_tour.append(nearest_station)
                    visited_cities.add(nearest_station)
                    distance_to_station = get_distance(current_tour[-2], nearest_station, location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df)
                    total_distance += distance_to_station
                    current_charge = max_charge
                    print("Total Distance After Charging Station Insertion:", total_distance)

        else:
            print(f"Impossible route encountered after {iteration_count} iterations. Restarting with a different source city.")
            new_source_city = places_df.loc[places_df["Locations"] != source_city, "Locations"].sample().values[0]
            return nearest_neighbor_algorithm(location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df, new_source_city, vehicle_params, charging_params, speed_dict, iteration_count + 1)

    # Check if the EV has enough charge to return to the source city
    distance_to_source_city = get_distance(current_tour[-1], source_city, location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df)
    speed_to_source_city = random.randint(50, 80)  # Generate random speed for the return trip
    discharge_rate_to_source_city = calculate_battery_discharge_rate(distance_to_source_city, speed_to_source_city, Cd, A, Crr, m, g, rho, lambda_, alpha)
    required_charge_to_source_city = discharge_rate_to_source_city

    if current_charge - required_charge_to_source_city >= min_charge:
        current_tour.append(source_city)
        total_distance += distance_to_source_city
        current_charge -= required_charge_to_source_city
        print(f"Returning to Source City: {source_city}")
        print(f"Total Distance After Returning to Source City: {total_distance}")
        print(f"Final Charge Level: {current_charge}")
    else:
        print("Warning: Insufficient charge to return to the source city. Looking for the nearest charging station.")
        nearest_station = None
        min_distance = float('inf')

        for station in charging_stations_df["Charging Stations"]:
            if station not in visited_cities:
                distance = get_distance(current_tour[-1], station, location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df)
                if distance < min_distance:
                    nearest_station = station
                    min_distance = distance

        if nearest_station is not None:
            current_tour.append(nearest_station)
            visited_cities.add(nearest_station)
            distance_to_station = get_distance(current_tour[-2], nearest_station, location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df)
            total_distance += distance_to_station
            current_charge = max_charge
            print("Total Distance After Charging Station Insertion:", total_distance)

            # Retry returning to the source city after charging
            current_tour.append(source_city)
            total_distance += distance_to_source_city
            current_charge -= required_charge_to_source_city
            print(f"Returning to Source City: {source_city}")
            print(f"Total Distance After Returning to Source City: {total_distance}")
            print(f"Final Charge Level: {current_charge}")
        else:
            print("Warning: No suitable charging station found to return to the source city.")

    print(f"Nearest Neighbor Algorithm completed after {iteration_count} iterations.")
    print("Final Tour:")
    for city in current_tour:
        if city in places_df["Locations"].values:
            print(f"Nearest Location: {city}")
        else:
            print(f"Nearest Charging Station: {city}")
    print(f"Total Distance: {total_distance}")

    return current_tour, total_distance, speed_dict
vehicle_params = {
    'Cd': 0.3,
    'A': 2.2,
    'Crr': 0.01,
    'm': 1500,
    'g': 9.81,
    'rho': 1.225,
    'lambda_': 0.80,
    'alpha': 0.25
}

charging_params = {
    'level2_charge_rate': 7,
    'dc_fast_charge_rate': 50,
    'min_charge': 20,  # 20% of 100 kWh battery capacity
    'max_charge': 90  # 90% of 100 kWh battery capacity
}

source_city = "Yale University"
speed_dict = {}  # Initialize an empty speed_dict
tour, total_distance, speed = nearest_neighbor_algorithm(location_distance_matrix, qualifying_distance_matrix, charging_station_distance_matrix, places_df, charging_stations_df, source_city, vehicle_params, charging_params, speed)

Speed from speed_dict: Yale University -> Sleeping Giant State Park: 40 miles/h
Current charge: 90, Required charge to next location: 1.2108994639665782
Updated Tour:
Nearest Location: Yale University
Nearest Location: Sleeping Giant State Park
Total Distance: 7.320760031537976
Updated Charge: 88.78910053603342
Speed: 40 miles/h
Speed from speed_dict: Sleeping Giant State Park -> Wharton Brook State Park: 40 miles/h
Current charge: 88.78910053603342, Required charge to next location: 0.5313209445879292
Updated Tour:
Nearest Location: Yale University
Nearest Location: Sleeping Giant State Park
Nearest Location: Wharton Brook State Park
Total Distance: 10.532978098194643
Updated Charge: 88.25777959144548
Speed: 40 miles/h
Speed from speed_dict: Wharton Brook State Park -> Hubbard Park: 40 miles/h
Current charge: 88.25777959144548, Required charge to next location: 1.4058416199637203
Updated Tour:
Nearest Location: Yale University
Nearest Location: Sleeping Giant State Park
Nearest Locati

In [17]:
def create_folium_graph(tour, places_df, charging_stations_df):
    # Create a map centered on the mean latitude and longitude of all locations
    map_center = (places_df["Latitude"].mean(), places_df["Longitude"].mean())
    folium_map = folium.Map(location=map_center, zoom_start=8)

    # Add markers for each location in the tour
    for city in tour:
        if city in places_df["Locations"].values:
            location_info = places_df[places_df["Locations"] == city]
            folium.Marker(
                location=[location_info["Latitude"].values[0], location_info["Longitude"].values[0]],
                popup=city,
                icon=folium.Icon(color='blue', icon='info-sign')
            ).add_to(folium_map)
        else:
            station_info = charging_stations_df[charging_stations_df["Charging Stations"] == city]
            folium.Marker(
                location=[station_info["Latitude"].values[0], station_info["Longitude"].values[0]],
                popup=city,
                icon=folium.Icon(color='green', icon='info-sign')
            ).add_to(folium_map)

    # Connect the locations and charging stations in the order of the tour
    locations = []
    for city in tour:
        if city in places_df["Locations"].values:
            location_info = places_df[places_df["Locations"] == city]
            locations.append([location_info["Latitude"].values[0], location_info["Longitude"].values[0]])
        else:
            station_info = charging_stations_df[charging_stations_df["Charging Stations"] == city]
            locations.append([station_info["Latitude"].values[0], station_info["Longitude"].values[0]])

    # Add the first location again to complete the cycle
    locations.append(locations[0])

    folium.PolyLine(
        locations=locations,
        color='red',
        weight=2,
        opacity=1
    ).add_to(folium_map)

    return folium_map

In [18]:
optimized_folium_map = create_folium_graph(tour, places_df, charging_stations_df)
optimized_folium_map